# Checking GPU available for trining

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# importing libraries

In [2]:
import random
import gradio as gr
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import string
import json
import time
import pandas as pd
import numpy as np

# loading the data

In [3]:
from tensorflow.keras.models import load_model
model = load_model('mychatbot.h5')

with open('covid_responses.json', 'rb') as f:
  responses = json.load(f)

data = pd.read_csv('data.csv')

tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
le = LabelEncoder()
le.fit_transform(data['tags'])

array([4, 4, 4, ..., 0, 0, 0])

# Building the output chatbot

In [4]:
def model_output(prediction_input):
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p = []
  texts_p.append(prediction_input)

  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],18)

  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()

  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  final_response = str(random.choice(responses[response_tag]))

  return final_response

# local testing

In [5]:
input = 'I am positive covid'
model_testing = model_output(input)
print('Chatbot: ' + model_testing)

Chatbot: Try to work from home if you can. If you are unable to work from home, talk to your employer about options available to you.


# testing with GUI

In [6]:
dialog_app = gr.Interface(model_output, 
                        gr.Textbox(placeholder="Enter your question"), 
                        "text",
                        examples=[["What is COVID-19?"],[ "What are the symptoms?"], ["What are the precautions?"], ["What is the treatment?"]],
                        title="COVID-19 Chatbot",
                        description="Ask your questions about COVID-19",
)
dialog_app.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
